In [1]:
from collections import namedtuple
import csv
import os
import sqlite3
import pandas as pd

import requests

DATA_URL = 'https://query.data.world/s/ezwk64ej624qyverrw6x7od7co7ftm'
DATA_CACHED = 'nba.data'
NBA_DB = 'nba.db'

In [40]:
# start clean
if os.path.isfile(NBA_DB):
    os.remove(NBA_DB)

Player = namedtuple('Player', ('name year first_year team college active '
                               'games avg_min avg_points'))

conn = sqlite3.connect(NBA_DB)
cur = conn.cursor()

In [3]:
def _get_csv_data():
    """GIVEN:
       Load in CSV data in from remote URL or local cache file"""
    if os.path.isfile(DATA_CACHED):
        with open(DATA_CACHED) as f:
            return f.read()
    else:
        with requests.Session() as session:
            return session.get(DATA_URL).content.decode('utf-8')


def load_data():
    """GIVEN:
       Converts NBA CSV data into a list of Player namedtuples"""
    content = _get_csv_data()
    reader = csv.DictReader(content.splitlines(), delimiter=',')
    for row in reader:
        player = Player(name=row['Player'],
                        year=row['Draft_Yr'],
                        first_year=row['first_year'],
                        team=row['Team'],
                        college=row['College'],
                        active=row['Yrs'],
                        games=row['Games'],
                        avg_min=row['Minutes.per.Game'],
                        avg_points=row['Points.per.Game'])
        yield player

# CODE HERE (tests under __main__):

In [41]:
cur.execute("""
    CREATE TABLE players (
        name, 
        year INTEGER, 
        first_year INTEGER, 
        team, 
        college,
        active INTEGER, 
        games INTEGER, 
        avg_min FLOAT, 
        avg_points FLOAT)""")

In [5]:
def import_to_db(players=None):
    """Create database table in sqlite3 and import the players data

       required table SQL:
       CREATE TABLE players (name, year, first_year, team, college,
                             active, games, avg_min, avg_points)
    """
    if players is None:
        players = list(load_data())
        players = list(load_data())
        cur.executemany("""INSERT INTO players 
                                  (name, year, first_year, team, college, active, games, avg_min, avg_points) 
                           VALUES (   ?,    ?,          ?,    ?,       ?,      ?,     ?,       ?,          ?)""", players)

In [42]:
import_to_db()
df = pd.read_sql("SELECT * FROM players", con=conn);
df.head()

,name,year,first_year,team,college,active,games,avg_min,avg_points
0,Robert Parish,1976,0,GSW,Centenary College of Louisiana,21,1611,28.4,14.5
1,Sonny Parker,1976,0,GSW,Texas A&M University,6,452,24.2,9.9
2,Marshall Rogers,1976,0,GSW,University of Texas-Pan American,1,26,6.8,3.8
3,Jeff Fosnes,1976,0,GSW,Vanderbilt University,0,0,0.0,0.0
4,Carl Bird,1976,0,GSW,University of California,0,0,0.0,0.0


In [7]:
df.describe()

,year,first_year,active,games,avg_min,avg_points
count,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000
mean,1989.792477,0.015148,3.266094,187.461500,9.325120,3.697122
std,11.500696,0.122156,4.614195,301.730508,10.921036,4.962079
min,1976.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1981.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1985.000000,0.000000,1.000000,8.000000,4.800000,1.400000
75%,1999.000000,0.000000,5.000000,270.000000,17.700000,6.100000
max,2015.000000,1.000000,21.000000,1611.000000,41.100000,30.100000


In [8]:
pd.read_sql("SELECT * FROM players ORDER BY avg_points DESC", con=conn).head()

,name,year,first_year,team,college,active,games,avg_min,avg_points
0,Michael Jordan,1984,0,CHI,University of North Carolina,15,1072,38.3,30.1
1,Kevin Durant,2007,0,OKC,University of Texas at Austin,9,641,37.8,27.4
2,LeBron James,2003,0,CLE,0,13,987,39.0,27.2
3,Allen Iverson,1996,0,PHI,Georgetown University,14,914,41.1,26.7
4,Kobe Bryant,1996,0,NOH,0,20,1346,36.1,25.0


In [9]:
def player_with_max_points_per_game():
    """The player with highest average points per game"""
    c = cur.execute("SELECT name FROM players ORDER BY avg_points DESC")
    r = c.fetchone()
    return r[0]
    
player_with_max_points_per_game()

'Michael Jordan'

In [10]:
def number_of_players_from_duke():
    """Return the number of players with college == Duke University"""
    c = cur.execute("SELECT COUNT(*) FROM players WHERE college = ?", ('Duke University',))
    r = c.fetchone()
    return r[0]

number_of_players_from_duke()

58

In [60]:
def percentage_of_players_first_year():
    """Return 2 digit percentage of players whose first year it is
       (first_year column)"""
    c = cur.execute("SELECT COUNT(*) FROM players")
    r = c.fetchone()
    total_count = r[0]
    c = cur.execute("SELECT COUNT(*) FROM players WHERE first_year = ?", (1,))
    r = c.fetchone()
    first_year_count = r[0]
    return first_year_count / total_count * 100.0

percentage_of_players_first_year()

1.5147689977278465

In [29]:
def avg_years_active_players_stanford():
    """Return the average years that players from "Stanford University
       are active ("active" column)"""
    c = cur.execute("SELECT AVG(active) FROM players WHERE college = ?", ('Stanford University',))
    r = c.fetchone()
    return r[0]

avg_years_active_players_stanford()

4.583333333333333

In [31]:
def year_with_most_drafts():
    """Return the year with the most drafts, in SQL you can use GROUP BY"""
    c = cur.execute("SELECT year, COUNT(*) FROM players GROUP BY year ORDER BY COUNT(*) DESC")
    r = c.fetchone()
    return r[0]

year_with_most_drafts()

1984

In [55]:
def most_games_per_year_for_veterans():
    """Top 6 players that are > 10 years active, that have the
       highest # games / year"""
    c = cur.execute("SELECT name FROM players WHERE active > 10 ORDER BY games/active DESC")
    top_size = []
    while (len(top_size) < 6):
        r = c.fetchone()
        top_size.append(r[0])
    return top_size

most_games_per_year_for_veterans()


['Terry Tyler',
 'John Stockton',
 'Mark Eaton',
 'A.C. Green',
 'Alex English',
 'Jack Sikma']

In [ ]:
if __name__ == '__main__':
    import_to_db()

    # A. check if the import went well
    def _verify_total_row_count_after_import():
        sql = '''SELECT COUNT(*) FROM players'''
        cur.execute(sql)
        ret = cur.fetchall()
        return ret[0][0]

    assert _verify_total_row_count_after_import() == 3961

    # B. some simple asserts of the data analysis functions
    assert player_with_max_points_per_game() == 'Michael Jordan'

    assert number_of_players_from_duke() == 58

    assert round(avg_years_active_players_stanford(), 2) == 4.58

    assert round(percentage_of_players_first_year(), 2) == 1.51

    assert int(year_with_most_drafts()) == 1984

    expected = ['A.C. Green', 'Alex English', 'Jack Sikma',
                'John Stockton', 'Mark Eaton', 'Terry Tyler']
    assert sorted(most_games_per_year_for_veterans()) == expected
